In [1]:
import pandas as pd
from pathlib import Path

In [20]:
md5_com = pd.read_csv("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/data2/md5sum_fromCompany.txt", sep='  ',  header=None, engine='python', names=['md5','name'])
md5_checked = pd.read_csv("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/data2/md5sum_checked.txt", sep='  ',  header=None, engine='python', names=['md5','name'])
merged = md5_com.merge(md5_checked, how='left', left_on='name', right_on='name', suffixes=['_company','_checked'])
merged

,md5_company,name,md5_checked
0,66caca97042a4f78e349edcf3f3b262f,31396-P5-WT-2_R1.fastq.gz,66caca97042a4f78e349edcf3f3b262f
1,2efabaafa39796e95bc4bea7dd262cf7,31396-P5-WT-2_R2.fastq.gz,2efabaafa39796e95bc4bea7dd262cf7
2,efcce9bf96356f568cc91a46a724c80d,Hela_sgSART3-KO__4__Pos_pCW-GFP-EV_M2__Replica...,efcce9bf96356f568cc91a46a724c80d
3,41e39a26439c3a69445f83690eda093c,Hela_sgSART3-KO__4__Pos_pCW-GFP-EV_M2__Replica...,41e39a26439c3a69445f83690eda093c
4,809f55a1394df31d40e07280db7c79ee,Hela_sgSART3-KO__4__Pos_pCW-GFP-SART3-W377A_Re...,809f55a1394df31d40e07280db7c79ee
...,...,...,...
109,5f24196b7ae4515ef5f4aa69d18added,RFH838-MRTXR-DMSO-R3_R2.fastq.gz,5f24196b7ae4515ef5f4aa69d18added
110,9d262ac969686bf68058301951496044,RFH838-MRXTR-MRTX-R1_R1.fastq.gz,9d262ac969686bf68058301951496044
111,e5809fedba983a01f0f6365abc74d156,RFH838-MRXTR-MRTX-R1_R2.fastq.gz,e5809fedba983a01f0f6365abc74d156
112,4b2db0dc1c48ea62ae128f164513a2cd,RFH838-MRXTR-MRTX-R3_R1.fastq.gz,4b2db0dc1c48ea62ae128f164513a2cd


In [21]:
(merged['md5_company'] == merged['md5_checked']).sum()

114

In [7]:
# from Bio.SeqRecord import SeqRecord
# from Bio import SeqIO

# def make_rc_record(record):
#     if "Mammalia" in record.description:
#         return SeqRecord(seq = record.seq.back_transcribe(), id = "DNA_" + record.id, description = record.description)
# records = map(make_rc_record, SeqIO.parse("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/arb-silva.de_2023-01-11_id1226795_tax_silva.fasta", "fasta"))
# records = list(filter(lambda item: item is not None, records))
# SeqIO.write(records, "rDNA_2.fasta", "fasta")


In [2]:
import json

with open('/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/qc/raw/multiqc_data/multiqc_data.json', "r") as f:
    data = json.load(f)
raw = pd.DataFrame(data['report_general_stats_data'][0]).T.sort_index()
raw['sample'] = raw.index
raw['sample'] = raw['sample'].apply(lambda x: x[:-3])
raw = raw.groupby("sample").aggregate({"percent_gc":"mean","avg_sequence_length":"mean","total_sequences":"mean"})
raw["total_sequences"] = raw["total_sequences"]*2

with open('/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/qc/qc_remove_adapter/multiqc_data/multiqc_data.json', "r") as f:
    data = json.load(f)
raw_adapter = pd.DataFrame(data['report_general_stats_data'][0]).T.sort_index()
raw_adapter['sample'] = raw_adapter.index
raw_adapter['sample'] = raw_adapter['sample'].apply(lambda x: x[:-9])
raw_adapter = raw_adapter.groupby("sample").aggregate({"percent_gc":"mean","avg_sequence_length":"mean","total_sequences":"mean"})
raw_adapter["total_sequences"] = raw_adapter["total_sequences"]*2

with open('/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/qc/data_remove_MT_rDNA/multiqc_data/multiqc_data.json', "r") as f:
    data = json.load(f)
raw_adapter_decontaminate = pd.DataFrame(data['report_general_stats_data'][0]).T.sort_index()
raw_adapter_decontaminate['sample'] = raw_adapter_decontaminate.index
raw_adapter_decontaminate['sample'] = raw_adapter_decontaminate['sample'].apply(lambda x: x[:-8])
raw_adapter_decontaminate = raw_adapter_decontaminate.groupby("sample").aggregate({"percent_gc":"mean","avg_sequence_length":"mean","total_sequences":"mean"})
raw_adapter_decontaminate["total_sequences"] = raw_adapter_decontaminate["total_sequences"]*2

In [3]:
combined = pd.concat([raw, raw_adapter['total_sequences'], raw_adapter_decontaminate['total_sequences']], axis=1)
combined['total_sequences'] = combined['total_sequences'].astype(int)
combined.columns = ['percent_gc','avg_sequence_length','total_sequences_raw','total_sequences_after_remove_adapter','total_sequences_after_decontamination']
combined = combined.sort_index()
combined

,percent_gc,avg_sequence_length,total_sequences_raw,total_sequences_after_remove_adapter,total_sequences_after_decontamination
sample,,,,,
21415-P5-OE-3,47.0,150.0,84947664,83686496,78212618
31284-P5-OE-2,47.5,150.0,80118494,78775618,74368124
31325-KI-2,46.5,150.0,85355886,83991982,79264066
31329-KI-1,46.5,150.0,91095542,90134442,84530184
31332-KI-3,45.5,150.0,83420596,82294576,78299566
31351-P5-OE-1,47.5,150.0,79077736,78329944,73999306
31371-KO-2,47.5,150.0,84629504,83527586,78933762
31372-KO-1,47.5,150.0,85243398,84041780,79568634
31373-KO-3,47.5,150.0,83299850,82118298,76940784


In [6]:
align_sum_a = {}
for f in Path("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/data/alignments_clean_TOPHAT").glob("*/align_summary.txt"):
    with open(f, "r") as file:
        align = [i[:4] for i in file.readlines() if " overall read mapping rate." in i]
        align_sum_a[f.parent.name] = float(align[0])
a = pd.DataFrame(align_sum_a, index=["TOPHAT"]).T

align_sum_b = {}
for f in Path("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/log/alignments_clean_HISAT").glob("*"):
    with open(f, "r") as file:
        align = [i[:5] for i in file.readlines() if " overall alignment rate" in i]
        align_sum_b[f.stem] = float(align[0])
b = pd.DataFrame(align_sum_b, index=["HISAT"]).T

align_sum_c = {}
for f in Path("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/new/data/alignments_clean_STAR").glob("*Log.final.out"):
    with open(f, "r") as file:
        align = [i.split("|")[1].strip()[:-1] for i in file.readlines() if "Uniquely mapped reads %" in i]
    with open(f, "r") as file:
        align_2 = [i.split("|")[1].strip()[:-1] for i in file.readlines() if "% of reads mapped to multiple loci" in i]
    align_sum_c[f.stem[:-9]] = float(align[0]) + float(align_2[0])
c = pd.DataFrame(align_sum_c, index=["STAR"]).T

results = combined.merge(a, how='left', right_index=True, left_index=True).merge(b, how='left', right_index=True, left_index=True).merge(c, how='left', right_index=True, left_index=True)
results.to_csv("new/alignments_results.csv")
results

,percent_gc,avg_sequence_length,total_sequences_raw,total_sequences_after_remove_adapter,total_sequences_after_decontamination,TOPHAT,HISAT,STAR
sample,,,,,,,,
21415-P5-OE-3,47.0,150.0,84947664,83686496,78212618,79.6,83.85,67.98
31284-P5-OE-2,47.5,150.0,80118494,78775618,74368124,80.7,83.52,70.21
31325-KI-2,46.5,150.0,85355886,83991982,79264066,83.1,83.04,59.87
31329-KI-1,46.5,150.0,91095542,90134442,84530184,82.7,85.19,66.25
31332-KI-3,45.5,150.0,83420596,82294576,78299566,81.4,80.95,54.45
31351-P5-OE-1,47.5,150.0,79077736,78329944,73999306,78.6,84.40,71.86
31371-KO-2,47.5,150.0,84629504,83527586,78933762,83.0,86.64,70.85
31372-KO-1,47.5,150.0,85243398,84041780,79568634,82.0,87.61,75.10
31373-KO-3,47.5,150.0,83299850,82118298,76940784,82.2,87.40,74.25
